In [53]:
import pandas as pd
import tiktoken
from collections import defaultdict
import numpy as np

In [54]:
df_base = pd.read_csv("../data/essay_correction_data.csv", delimiter=";", encoding='utf-8')
df_base

,prompt,description,title,text,comments,criteria_score_1,criteria_score_2,criteria_score_3,criteria_score_4,criteria_score_5
0,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Natureza caótica,A natureza possui um equilíbrio singular singu...,"Texto muito bom, com alguns erros e equívocos ...",160,200,160,200,160
1,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,Em busca de um novo lar,"Com o aumento do aquecimento global, várias es...",Texto mediano. Além dos problemas pontuais neg...,120,120,120,120,120
2,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,A ação antrópica e suas consequências,"Há, na pré-história, pinturas rupestres que re...","Texto prolixo e repetitivo, com problemas pont...",120,120,80,120,120
3,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,O aquecimento global e os ursos,O aquecimento global vem afetando o mundo nos ...,"Lamentavelmente, o texto é fraco, insuficiente...",120,80,80,80,80
4,Os ursos polares da Rússia e um dilema ecológico,As mudanças que afetam o clima de nosso planet...,As consequências do degelo no Ártico,"O aumento do aquecimento global provocou, no í...",Texto mediano. O autor cumpre as exigências da...,80,120,120,120,120
...,...,...,...,...,...,...,...,...,...,...
3721,A cultura do cancelamento nas redes sociais,A cultura do cancelamento tem se tornado cada ...,A Cultura do Cancelamento Deve Ser Cancelada,A cultura do cancelamento surgiu com o intuito...,As discussões precisam ser delimitadas e mais ...,120,160,160,200,200
3722,A cultura do cancelamento nas redes sociais,A cultura do cancelamento tem se tornado cada ...,A cultura do cancelamento e suas consequências,"Imersos na era da Revolução Industrial 4.0, on...","As discussões precisam ser mais exploradas, po...",160,160,160,200,160
3723,A cultura do cancelamento nas redes sociais,A cultura do cancelamento tem se tornado cada ...,A cultura do cancelamento,Julgamento. Desproporcionalidade. Criticidade....,"As discussões precisam ser mais exploradas, po...",160,160,160,200,200
3724,A cultura do cancelamento nas redes sociais,A cultura do cancelamento tem se tornado cada ...,A Cultura do cancelamento,"Com o aumento do uso da redes sociais, novos t...","Explore mais as discussões, pois a abordagem d...",160,160,120,200,160


In [55]:
def format_to_chatgpt_jsonl(essay_row):

    essay_prompt = "Tema da redação: " + essay_row["prompt"] + "."
    essay_title = "Título: " + essay_row["title"] + "."
    essay_text = "Redação: " + essay_row["text"] + "."

    essay_criteria_grade_1 = (
        "Competência 1: " + str(essay_row["criteria_score_1"]) + "."
    )
    essay_criteria_grade_2 = (
        "Competência 2: " + str(essay_row["criteria_score_2"]) + "."
    )
    essay_criteria_grade_3 = (
        "Competência 3: " + str(essay_row["criteria_score_3"]) + "."
    )
    essay_criteria_grade_4 = (
        "Competência 4: " + str(essay_row["criteria_score_4"]) + "."
    )
    essay_criteria_grade_5 = (
        "Competência 5: " + str(essay_row["criteria_score_5"]) + "."
    )

    essay_grades = " ".join(
        [
            essay_criteria_grade_1,
            essay_criteria_grade_2,
            essay_criteria_grade_3,
            essay_criteria_grade_4,
            essay_criteria_grade_5,
        ]
    )

    essay_review_comments = "Comentários: " + essay_row["comments"] + "."

    return {
        "messages": [
            {
                "role": "system",
                "content": "Shenem é um assistente de professores de redação que avalia e revisa os textos em português brasileiro.",
            },
            {
                "role": "user",
                "content": f"{essay_prompt + " " + essay_title + " " + essay_text}",
            },
            {
                "role": "assistant",
                "content": f"{essay_grades + ' ' + essay_review_comments}",
            },
        ]
    }


resultados = [format_to_chatgpt_jsonl(row) for _, row in df_base.iterrows()]

In [56]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str) -> int:
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(str(resultados))

3162863

In [62]:
def check_errors_in_dataset(dataset):
    # Format error checks
    format_errors = defaultdict(int)

    for ex in dataset:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(
                k not in ("role", "content", "name", "function_call", "weight")
                for k in message
            ):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in (
                "system",
                "user",
                "assistant",
                "function",
            ):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            function_call = message.get("function_call", None)

            if (not content and not function_call) or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found")


def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens


def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens


def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


def token_metadata_and_pricing(dataset):
    # Warnings and tokens counts
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 4096 for l in convo_lens)
    print(
        f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning"
    )

    # Pricing and default n_epochs estimate
    MAX_TOKENS_PER_EXAMPLE = 4096

    TARGET_EPOCHS = 3
    MIN_TARGET_EXAMPLES = 100
    MAX_TARGET_EXAMPLES = 25000
    MIN_DEFAULT_EPOCHS = 1
    MAX_DEFAULT_EPOCHS = 25

    n_epochs = TARGET_EPOCHS
    n_train_examples = len(dataset)
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

    n_billing_tokens_in_dataset = sum(
        min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens
    )
    print(
        f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training"
    )
    print(f"By default, you'll train for {n_epochs} epochs on this dataset")
    print(
        f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens"
    )

    # As of 28/04/2024 - https://openai.com/pricing
    # $8 per 1M tokens
    # $0.008 per 1K token
    print(
        f"By default, this dataset will cost ~${(n_epochs * n_billing_tokens_in_dataset * 0.008) / 1000} to train on"
    )


check_errors_in_dataset(resultados)
token_metadata_and_pricing(resultados)

No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 336, 1827
mean / median: 817.7184648416533, 810.0
p5 / p95: 617.5, 1015.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 92, 1045
mean / median: 199.2541599570585, 180.0
p5 / p95: 137.0, 241.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning
Dataset has ~3046819 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~9140457 tokens
By default, this dataset will cost ~$73.123656 to train on
